## CTAB-GAN generated dataset
- loan_status postprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [3]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Real_Datasets/train_category.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Real_Datasets/test_category.csv"
test_data = pd.read_csv(test_data_path)

## Load original data
fake_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/CTAB-GAN/Fake_Datasets/ctab-gan.csv"
fake = pd.read_csv(fake_path, low_memory=False)

In [4]:
for col in fake.columns:
    print(f"{col} features \n: {fake[col].unique()}\n")

last_fico_range_high features 
: [532.21606445 642.27423096 529.52124023 ... 500.1862793  566.94403076
 530.06713867]

annual_inc features 
: [84820.57664722 24661.91849181 47962.12870302 ... 77508.39958574
 37965.92147343 58253.56037027]

dti features 
: [19.94886208 17.06970406 18.8851738  ... 14.41803932 28.22223091
  3.71097231]

mo_sin_old_rev_tl_op features 
: [277.94955444  98.75395966 205.88439941 ... 128.61247253 177.06724548
  63.26057053]

revol_util features 
: [0.11111406 0.92269766 0.43376863 ... 0.45118776 0.5376693  0.17790577]

int_rate features 
: [0.28521153 0.14235406 0.19870087 ... 0.18043435 0.06837586 0.14165766]

installment features 
: [6.21457092 3.97627316 5.50695043 ... 5.66605378 5.82893218 6.24951101]

avg_cur_bal features 
: [23771.91455328 12301.48085681 12998.28899561 ...  5267.75170093
  1707.86948872  3269.85967795]

revol_bal features 
: [21508.01561115 29514.86199419 11001.84265141 ... 18246.84432118
  6007.38256569  3780.31887565]

debt_settlement_

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116458 entries, 0 to 1116457
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   last_fico_range_high  1116458 non-null  float64
 1   annual_inc            1116458 non-null  float64
 2   dti                   1116458 non-null  float64
 3   mo_sin_old_rev_tl_op  1116458 non-null  float64
 4   revol_util            1116458 non-null  float64
 5   int_rate              1116458 non-null  float64
 6   installment           1116458 non-null  float64
 7   avg_cur_bal           1116458 non-null  float64
 8   revol_bal             1116458 non-null  float64
 9   debt_settlement_flag  1116458 non-null  object 
 10  sub_grade             1116458 non-null  object 
 11  home_ownership        1116458 non-null  object 
 12  purpose               1116458 non-null  object 
 13  grade                 1116458 non-null  object 
 14  total_pymnt           1116458 non-

In [5]:
fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540000 entries, 0 to 539999
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   last_fico_range_high  540000 non-null  float64
 1   annual_inc            540000 non-null  float64
 2   dti                   540000 non-null  float64
 3   mo_sin_old_rev_tl_op  540000 non-null  float64
 4   revol_util            540000 non-null  float64
 5   int_rate              540000 non-null  float64
 6   installment           540000 non-null  float64
 7   avg_cur_bal           540000 non-null  float64
 8   revol_bal             540000 non-null  float64
 9   debt_settlement_flag  540000 non-null  object 
 10  sub_grade             540000 non-null  object 
 11  home_ownership        540000 non-null  object 
 12  purpose               540000 non-null  object 
 13  grade                 540000 non-null  object 
 14  total_pymnt           540000 non-null  float64
 15  

In [6]:
fake['loan_status'] = 1

fake['loan_status'].value_counts()

loan_status
1    540000
Name: count, dtype: int64